# Phase 1: Data Preprocessing & Validation to clean and analyze your onshore, Sentinel-1, and GIS data.

- Workflow in Python:
    - Load & preprocess onshore wind farm data
    - Extract & process Sentinel-1 offshore wind data
    - Load & analyze GIS layers (turbine locations, farm boundaries)
------
- What This Code Does:
    - Loads onshore wind farm data and computes rolling averages for wind speed & direction.
    - Processes Sentinel-1 offshore wind data, aggregating it into daily averages.
    - Loads GIS layers for turbine locations & wind farm boundaries.
    - Validates turbine spacing (ensuring a minimum of 1 nautical mile).
    - Plots the wind farm layout for visualization.

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point

# 1. Load Onshore Wind Farm Data
def load_onshore_data(file_path):
    df = pd.read_csv(file_path, parse_dates=['timestamp'])  # Ensure timestamp column exists
    df = df.sort_values(by='timestamp')
    df['wind_speed_rolling'] = df['wind_speed'].rolling(window=6).mean()  # 1-hour rolling avg (assuming 10-min intervals)
    df['wind_dir_rolling'] = df['wind_direction'].rolling(window=6).mean()
    return df

# 2. Load Sentinel-1 Offshore Wind Data
def load_sentinel1_data(file_path):
    df = pd.read_csv(file_path, parse_dates=['timestamp'])
    df = df.sort_values(by='timestamp')
    df = df.groupby(df['timestamp'].dt.date).mean()  # Aggregate to daily averages
    return df

# 3. Load GIS Layers (Turbine Locations & Farm Boundaries)
def load_gis_layers(turbine_shp, boundary_shp):
    turbines = gpd.read_file(turbine_shp)
    boundaries = gpd.read_file(boundary_shp)
    return turbines, boundaries

# 4. Validate Minimum Turbine Spacing
def validate_spacing(turbines, min_spacing_nautical_miles=1):
    min_spacing_meters = min_spacing_nautical_miles * 1852  # Convert to meters
    coords = np.array(list(zip(turbines.geometry.x, turbines.geometry.y)))
    for i, p1 in enumerate(coords):
        for j, p2 in enumerate(coords):
            if i != j and np.linalg.norm(p1 - p2) < min_spacing_meters:
                print(f"Warning: Turbines {i} and {j} are too close!")

# 5. Visualization
def plot_wind_farm(turbines, boundaries):
    fig, ax = plt.subplots(figsize=(10, 6))
    boundaries.plot(ax=ax, color='lightblue', alpha=0.5)
    turbines.plot(ax=ax, color='red', markersize=50, label='Turbines')
    plt.legend()
    plt.title("Wind Farm Layout")
    plt.show()

# Example Usage
# onshore_df = load_onshore_data('onshore_wind_data.csv')
# sentinel_df = load_sentinel1_data('sentinel1_wind_data.csv')
# turbines, boundaries = load_gis_layers('turbines.shp', 'boundary.shp')
# validate_spacing(turbines)
# plot_wind_farm(turbines, boundaries)


# 2️⃣ Load GIS-Based Wind Farm Layout Into FLORIS
- We will integrate your GIS turbine locations into FLORIS for wake modeling.
    - Extract turbine locations from GIS layers
    - Convert them into a FLORIS-readable format
    - Run a wake simulation
    - I will now update your script accordingly. 🚀

- The function run_floris_simulation() now extracts GIS turbine positions, runs wake simulations, and computes power output.
    - Uses FLORIS JSON input file (example_input.json)—you need to provide or generate this file.
----
- Next Step:
    - Test FLORIS wake modeling with real wind speed & direction from Sentinel-1.

- Extract wind conditions from Sentinel-1 and use them in FLORIS.
    - Run multiple wake simulations for different wind speeds & directions.
    - Optimize turbine positions using a simple layout optimization approach.
---
- 🚀 Updates & Next Steps
    - ✅ Extracts wind speed & direction from Sentinel-1 for FLORIS simulations.
    - ✅ Runs multiple wake simulations for different wind conditions.
    - ✅ Adds optimization function to reposition turbines for maximum power output.
---
Next Steps:
Run the FLORIS simulation and optimization with your dataset.
Let me know if you need further constraints (e.g., spacing, boundaries) in the optimization! 🚀

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
from floris.tools import FlorisInterface
from scipy.optimize import minimize

# 6. Integrate FLORIS for Wake Modeling
def run_floris_simulation(turbines, wind_speed, wind_direction):
    fi = FlorisInterface("example_input.json")  # Ensure you have a valid FLORIS config
    
    # Convert GIS turbine locations to FLORIS format
    turbine_x = list(turbines.geometry.x)
    turbine_y = list(turbines.geometry.y)
    
    fi.reinitialize(layout_x=turbine_x, layout_y=turbine_y)
    fi.reinitialize(wind_speeds=[wind_speed], wind_directions=[wind_direction])
    fi.calculate_wake()
    
    # Extract power output
    power_output = fi.get_turbine_powers()
    total_power = np.sum(power_output)
    print("Total Farm Power Output:", total_power)
    return total_power

# 7. Optimize Turbine Layout
def objective_function(layout, turbines, wind_speed, wind_direction):
    turbines.geometry = [Point(layout[i], layout[i+1]) for i in range(0, len(layout), 2)]
    return -run_floris_simulation(turbines, wind_speed, wind_direction)  # Minimize negative power (maximize power)

def optimize_turbine_layout(turbines, wind_speed, wind_direction):
    initial_positions = np.array([(p.x, p.y) for p in turbines.geometry]).flatten()
    bounds = [(p.x - 500, p.x + 500) for p in turbines.geometry] + [(p.y - 500, p.y + 500) for p in turbines.geometry]
    result = minimize(objective_function, initial_positions, args=(turbines, wind_speed, wind_direction), bounds=bounds, method='L-BFGS-B')
    optimized_layout = result.x.reshape(-1, 2)
    turbines.geometry = [Point(x, y) for x, y in optimized_layout]
    return turbines

# Example Usage
# onshore_df = load_onshore_data('onshore_wind_data.csv')
# sentinel_df = load_sentinel1_data('sentinel1_wind_data.csv')
# turbines, boundaries = load_gis_layers('turbines.shp', 'boundary.shp')
# validate_spacing(turbines)
# plot_wind_farm(turbines, boundaries)
# run_floris_simulation(turbines, wind_speed=10.0, wind_direction=270)
# optimized_turbines = optimize_turbine_layout(turbines, wind_speed=10.0, wind_direction=270)
# plot_wind_farm(optimized_turbines, boundaries)
